## How to run inference with a PyTorch time series Transformer
- https://towardsdatascience.com/how-to-run-inference-with-a-pytorch-time-series-transformer-394fd6cbe16c

<div style="text-align: right"> <b>Author : Kwang Myung Yu</b></div>
<div style="text-align: right"> Initial upload: 2023.11.03</div>
<div style="text-align: right"> Last update: 2023.11.03</div>

In [1]:
import datetime
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline
# print(plt.stype.available)

# Options for pandas
pd.options.display.max_columns = 30

이 글에서는 시계열 예측을 위해 파이토치 트랜스포머로 추론을 실행하는 방법을 보여드리겠습니다.   
특히, 이전 포스트인 파이토치로 시계열 예측을 위한 트랜스포머 만드는 법에서 설명한 파이토치 시계열 트랜스포머를 사용하겠습니다.

포스트의 구성은 다음과 같습니다:   
- 먼저, 파이토치 시계열 트랜스포머에 필요한 입력이 무엇인지 간략하게 설명하겠습니다.   
- 그런 다음 디코더 입력 값을 모를 때 모델을 사용하여 추론을 실행하는 방법을 보여드리겠습니다.  
- 마지막으로 이 접근법의 몇 가지 단점을 지적하겠습니다.

### 트랜스포머 모델에 필요한 입력

The transformer model requires the following inputs:   
- src: 인코더에서 사용. src의 형태는 [배치 크기, n, 입력 피처 수] 또는 [n, 배치 크기, 입력 특징 수](batch_first 생성자 인자의 값에 따라 다름)여야 하며, 여기서 n은 입력 계열의 데이터 포인트 수입니다. 예를 들어 시간별 전기 요금을 예측하는 경우 지난 주 데이터를 기반으로 예측을 하려면 n=168이 됩니다.   
- tgs: 디코더에서 사용됩니다. tgt는 src에 있는 입력 시퀀스의 마지막 값과 타겟 시퀀스의 마지막 값을 제외한 모든 값으로 구성됩니다. 즉, [배치 크기, m, 예측 변수 수] 또는 [m, 배치 크기, 예측 변수 수]의 형태를 가지게 되며, 여기서 m은 forecasting입니다. 전기 가격 예측의 예를 계속 이어서, 전기 가격을 48시간 전에 예측하려는 경우 m=48이 됩니다.

또한 인코더와 디코더에는 마스크가 필요합니다. 마스킹에 대한 소개는 위에 언급된 게시물을 참조하시기 바랍니다.

계속 진행하기 전에 주의해야 할 또 다른 사항은 이 블로그 게시물에서 사용하는 특정 시계열 트랜스포머 구현은 항상 [배치 크기, m, 예측 변수 수] 또는 [m, 배치 크기, 예측 변수 수] 형태의 텐서,  
즉 모델 출력 시퀀스의 길이가 tgt 텐서에서 디코더에 주어진 입력 시퀀스의 길이에 의해 결정된다는 점입니다.   

따라서 tgt의 모양이 [72, 배치 크기, 1]이면 tgt의 시퀀스 길이가 72이므로 모델에서도 72의 시퀀스를 출력한다는 의미입니다.

### How to use a time series Transformer for inference

자, 이제 사전 준비를 마쳤으니 시계열 예측을 위해 Transformer로 추론을 실행하는 방법에 대한 블로그 게시물이 왜 존재하는지 생각해 보겠습니다:   

훈련 중에는 목표 시퀀스의 값을 알고 있기 때문에 tgt를 생성하는 것이 간단합니다.  
그러나 프로덕션 환경과 같이 추론하는 동안에는 당연히 예측을 할 때 타겟 시퀀스의 값을 알지 못합니다.   
그렇지 않다면 애초에 예측을 할 필요가 없기 때문입니다.   
따라서 추론 중에 모델에 입력으로 사용할 수 있는 합리적인 TGT를 생성하는 방법을 찾아야 합니다.

이제 시계열 트랜스포머에 어떤 입력이 필요한지, 어떻게든 tgt를 생성해야 하는 이유를 알았으니 실제로 어떻게 생성하는지 살펴보겠습니다.  
다음에서는 전체적인 목적은 tgt 텐서를 생성하는 것이며, 생성된 텐서는 예측을 위한 모델의 입력으로 사용할 수 있다는 점을 명심하세요.

간단한 예를 들어 설명하기 위해 추론 시점인 t에서 시퀀스의 가장 최근 관측치 5개를 기반으로 시퀀스의 다음 3개 값을 예측하고자 한다고 가정해 보겠습니다.

src가 다음과 같다고 가정하자.   

src = [xt-4, xt-3, xt-2, xt-1, xt]

목표는 tgt_y를 예측하는 것입니다:  

tgt_y = [xt+1, xt+2, xt+3]

따라서 모델이 tgt_y에 대한 예측을 수행하기 위해 입력으로 필요한 tgt 는 다음과 같아야 합니다:  

tgt = [xt, xt+1, xt+2].

xt의 값은 알지만 xt+1이나 xt+2의 값은 모르기 때문에 어떻게든 이를 추정해야 합니다.   
이 글에서는 먼저 xt+1 을 예측한 다음, 이 예측값을 tgt에 추가하여 tgt = [xt, xt+1]이 되도록 한 다음,  
이 tgt를 사용하여 xt+2 를 예측한 다음,  
이 예측값을 tgt에 추가하여 tgt = [xt, xt+1, xt+2]가 되도록 하고 마지막으로 이 tgt를 사용하여 최종 예측을 생성하는 방식으로 이 작업을 수행하겠습니다.

아래 함수는 파이토치에서 시계열 트랜스포머 모델로 추론을 실행하는 데 필요한 코드입니다.  
이 함수는 위에서 설명한 접근 방식에 따라 예측을 생성합니다.  
문서에 설명된 몇 가지 다른 인수와 함께 Transformer 모델과 src를 전달합니다.  
그런 다음 이 함수는 tgt를 반복적으로 생성하고 t 시점에 마지막으로 알려진 관측치와 나머지 m-1 데이터 포인트에 대한 추정 값으로 구성된 tgt를 기반으로 최종 예측을 생성합니다.

In [2]:
import torch.nn as nn 
import torch
import utils

In [3]:
def run_encoder_decoder_inference(
    model: nn.Module, 
    src: torch.Tensor, 
    forecast_window: int,
    batch_size: int,
    device,
    batch_first: bool=False
    ) -> torch.Tensor:

    """
    NB! 이 함수는 현재 batch_first = False로 작동하는 모델에서만 테스트됩니다.
    
    이 함수는 인코더-디코더 유형 모델에서 디코더가 훈련 중에 목표 시퀀스인 tgt라는 입력을 필요로 하는 경우에 사용됩니다. 
    추론 중에는 tgt의 값을 알 수 없으므로 값을 반복적으로 생성해야 합니다.  
    
    이 함수는 src의 각 배치에 대한 길이 forecast_window의 예측을 반환합니다.
    
    NB! 그라디언트 계산 없이 추론을 수행하려면 컨텍스트 관리자 내에서 이 함수를 호출해야 합니다. 

    torch.no_grad처럼:
       와 같이 호출하세요:
        RUN_ENCODER_DECODER_INFERENCE()
        
    컨텍스트 매니저는 이 함수 내에서 의도적으로 호출되지 않도록 하여 훈련 및 기울기 계산 중에 역전파되어야 하는 손실을 계산하는 데 
    사용되는 경우에 사용할 수 있도록 합니다.
    
    
    use_predicted_tgt = True인 경우:
    우선, tgt는 src의 마지막 값과 같습니다. 
    그런 다음 모델 예측의 마지막 요소는 반복적으로 tgt와 연결되어 for-loop의 각 단계에서 tgt의 크기가 1씩 증가합니다. 
    마지막으로 tgt는 올바른 길이(목표 시퀀스 길이)를 가지게 되고 최종 예측이 생성되어 반환됩니다.
    
    Args:
        model: An encoder-decoder type model where the decoder requires
               target values as input. Should be set to evaluation mode before 
               passed to this function.
               
        src: The input to the model
        
        forecast_horizon: The desired length of the model's output, e.g. 58 if you
                         want to predict the next 58 hours of FCR prices.
                           
        batch_size: batch size
        
        batch_first: If true, the shape of the model input should be 
                     [batch size, input sequence length, number of features].
                     If false, [input sequence length, batch size, number of features]
    
    """

    # Dimension of a batched model input that contains the target sequence values
    target_seq_dim = 0 if batch_first == False else 1

    # Take the last value of the target variable in all batches in src and make it tgt
    # as per the Influenza paper
    tgt = src[-1, :, 0] if batch_first == False else src[:, -1, 0] # shape [1, batch_size, 1]

    # Change shape from [batch_size] to [1, batch_size, 1]
    if batch_size == 1 and batch_first == False:
        tgt = tgt.unsqueeze(0).unsqueeze(0) # change from [1] to [1, 1, 1]

    # Change shape from [batch_size] to [1, batch_size, 1]
    if batch_first == False and batch_size > 1:
        tgt = tgt.unsqueeze(0).unsqueeze(-1)

    # Iteratively concatenate tgt with the first element in the prediction
    for _ in range(forecast_window-1):

        # Create masks
        dim_a = tgt.shape[1] if batch_first == True else tgt.shape[0]

        dim_b = src.shape[1] if batch_first == True else src.shape[0]

        tgt_mask = utils.generate_square_subsequent_mask(
            dim1=dim_a,
            dim2=dim_a,
            device=device
            )

        src_mask = utils.generate_square_subsequent_mask(
            dim1=dim_a,
            dim2=dim_b,
            device=device
            )

        # Make prediction
        prediction = model(src, tgt, src_mask, tgt_mask) 

        # If statement simply makes sure that the predicted value is 
        # extracted and reshaped correctly
        if batch_first == False:

            # Obtain the predicted value at t+1 where t is the last time step 
            # represented in tgt
            last_predicted_value = prediction[-1, :, :] 

            # Reshape from [batch_size, 1] --> [1, batch_size, 1]
            last_predicted_value = last_predicted_value.unsqueeze(0)

        else:

            # Obtain predicted value
            last_predicted_value = prediction[:, -1, :]

            # Reshape from [batch_size, 1] --> [batch_size, 1, 1]
            last_predicted_value = last_predicted_value.unsqueeze(-1)

        # Detach the predicted element from the graph and concatenate with 
        # tgt in dimension 1 or 0
        tgt = torch.cat((tgt, last_predicted_value.detach()), target_seq_dim)
    
    # Create masks
    dim_a = tgt.shape[1] if batch_first == True else tgt.shape[0]

    dim_b = src.shape[1] if batch_first == True else src.shape[0]

    tgt_mask = utils.generate_square_subsequent_mask(
        dim1=dim_a,
        dim2=dim_a,
        device=device
        )

    src_mask = utils.generate_square_subsequent_mask(
        dim1=dim_a,
        dim2=dim_b,
        device=device
        )

    # Make final prediction
    final_prediction = model(src, tgt, src_mask, tgt_mask)

    return final_prediction


이 함수는 유효성 검사 또는 테스트 루프 내에서 사용하도록 설계되었습니다. 모델을 호출하여 예측을 생성하는 대신 추론 함수를 호출합니다. 다음은 사용 방법에 대한 간단한 예입니다:

In [4]:
import torch
import inference
import utils

```python
epochs = 10
forecast_window = 48 # supposing you're forecasting 48 hours ahead
enc_seq_len = 168 # supposing you want the model to base its forecasts on the previous 7 days of data

optimizer = torch.optim.Adam()

criterion = torch.nn.MSELoss()
```

```python
# Iterate over all epochs
for epoch in range(epochs):

    # Iterate over all (x,y) pairs in training dataloader
    for i, (src, tgt, tgt_y) in enumerate(training_dataloader):

        # zero the parameter gradients
        optimizer.zero_grad()

        # Generate masks
        tgt_mask = utils.generate_square_subsequent_mask(
            dim1=forecast_window,
            dim2=forecast_window
            )

        src_mask = utils.generate_square_subsequent_mask(
            dim1=forecast_window,
            dim2=enc_seq_len
            )

        # Make forecasts
        prediction = model(src, tgt, src_mask, tgt_mask)

        # Compute and backprop loss
        loss = criterion(tgt_y, prediction)

        loss.backward()

        # Take optimizer step
        optimizer.step()


    # Iterate over all (x,y) pairs in validation dataloader
    model.eval()

    with torch.no_grad():
    
        for i, (src, _, tgt_y) in enumerate(validation_dataloader):

            prediction = inference.run_encoder_decoder_inference(
                model=model, 
                src=src, 
                forecast_window=forecast_window,
                batch_size=src.shape[1]
                )

            loss = criterion(tgt_y, prediction)
```

이 스크립트를 그대로 사용할 수 없다는 점에 유의하세요.  
이 스크립트는 전체적인 아이디어를 보여주기 위한 예제일 뿐이며,  
복사하여 붙여넣기만 하면 작동할 것으로 기대할 수 있는 것이 아닙니다.  
예를 들어, 스크립트를 작동시키려면 먼저 모델과 데이터 로더를 인스턴스화해야 합니다.  
이 블로그 게시물의 GitHub 리포지토리에 있는 sandbox.py 파일에서 이 작업을 수행하는 방법에 대한 예제를 참조하세요.  
PyTorch 신경망을 훈련하고, 검증하고, 테스트해 본 적이 없다면 PyTorch의 초보자 수준 튜토리얼을 살펴보는 것이 좋습니다.

### Downsides of the shown approach for running inference with a time series Transformer

추론 함수가 반복적으로 tgt 를 생성하기 위해 루프에 의존한다는 점을 감안할 때, m이 크면 루프의 반복 횟수가 증가하기 때문에 함수가 느려질 수 있습니다. 이것이 위에서 설명한 접근 방식의 주요 단점입니다.  
제가 상상력이 부족해서 더 효율적인 접근 방식을 생각해내지 못했지만,  
아이디어가 있으시면 댓글 섹션에서 여러분의 의견을 듣고 싶습니다. 리포지토리에 직접 기여하는 것도 환영합니다.

추론 함수가 배치당 모델을 m-1회 호출한다는 점을 감안할 때,  
매개 변수가 많은 모델을 사용하거나 큰 n을 사용하는 등 모델을 호출하는 계산 시간을 증가시키는 몇 가지 사항에 주의해야 할 수 있습니다.  
또한 배치가 많을수록 추론 함수가 호출되는 횟수가 많아지고 전체 훈련 또는 테스트 스크립트를 실행하는 데 더 오래 걸리게 됩니다.